In [170]:
import re
import pandas as pd
import numpy as np
import nltk
import pymorphy3
import optuna
import mlflow

from optuna.integration.mlflow import MLflowCallback
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import ndcg_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.sparse import csr_matrix, vstack
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from gensim.utils import simple_preprocess

import warnings
warnings.simplefilter('ignore', FutureWarning)

In [105]:
RANDOM_STATE = 42

In [103]:
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "hr-ai-scout"

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# Load data

In [7]:
df = pd.read_csv('/Users/user/Documents/Magistracy/yearly_project/hr-ai-scout/total_df.csv')
df.head()

,vacancy_id,vacancy_name,vacancy_area,vacancy_experience,vacancy_employment,vacancy_schedule,vacancy_salary_from,vacancy_salary_to,vacancy_salary_currency,vacancy_salary_gross,...,resume_education,resume_courses,resume_salary,resume_age,resume_total_experience,resume_experience_months,resume_location,resume_gender,resume_applicant_status,target
0,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,300000.0,NaN,RUR,False,...,['Казанский Авиационный Институт'],NaN,NaN,65.0,19 лет,228.0,Москва,Мужчина,Рассматривает предложения,1
1,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,300000.0,NaN,RUR,False,...,"['ООО ""Открытый Учебный Центр СофтБаланс"", г. ...","['ООО ""Открытый Учебный Центр СофтБаланс"", г. ...",NaN,43.0,17 лет 4 месяца,208.0,Москва,Мужчина,Рассматривает предложения,1
2,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,300000.0,NaN,RUR,False,...,['Орский государственный педагогический инстит...,NaN,200 000 ₽ на руки,52.0,30 лет,360.0,Москва,Женщина,NaN,1
3,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,300000.0,NaN,RUR,False,...,['Красноярский государственный университет'],NaN,500 000 ₽ на руки,56.0,29 лет 8 месяцев,356.0,Красноярск,Мужчина,Рассматривает предложения,1
4,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,300000.0,NaN,RUR,False,...,['Белоруский Гос. Университет Информатики и Ра...,"['SAP CIS, SAP XI', 'Школа Логистики МАДИ', 'S...",NaN,48.0,25 лет 1 месяц,301.0,Moscow,Male,NaN,1


# Preprocessing

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">
В первую очередь уберем строки, где пропущены все ключевые значения в резюме:
</div>

In [8]:
t1 = df.shape[0]
df = df.dropna(subset= ["resume_education",
                        "resume_last_experience_description",
                        "resume_last_position",
                        "resume_last_company_experience_period",
                        "resume_total_experience",
                        "resume_experience_months",
                        "resume_location",
                        "resume_specialization",
                        # "resume_gender",
                        # "resume_title"
                       ], how="all")
t2 = df.shape[0]
print('Удалено ', t1 - t2 ,' строки')

Удалено  84  строки


<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">
Удалим еще те строки, где случился технический сбой в парсинге, где у кандидата общий опыт есть, а последний опыт не указан (и наоборот):
</div>

In [9]:
t1 = df.shape[0]
df = df.loc[~(df["resume_total_experience"].notna()
        & df["resume_last_experience_description"].isna()
        & df["resume_last_position"].isna())]
t2 = df.shape[0]
print('Удалено ', t1 - t2 ,' строк')

Удалено  1543  строк


In [10]:
t1 = df.shape[0]
df = df.loc[~(df["resume_total_experience"].isna()
        & df["resume_last_experience_description"].notna()
        & df["resume_last_position"].notna())]
t2 = df.shape[0]
print('Удалено ', t1 - t2 ,' строк')

Удалено  0  строк


<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">
Посмотрим на пропуски отдельно по категориальным и числовым признакам.
</div>

In [11]:
num_cols = df.select_dtypes(include=[np.number]).columns
cat_cols = df.select_dtypes(include=['object']).columns

In [12]:
df[cat_cols] = df[cat_cols].fillna('NDT')

In [13]:
df.loc[df['resume_experience_months'].isna(), 'resume_last_experience_description'].unique()

array(['NDT'], dtype=object)

In [14]:
df['resume_age'] = df['resume_age'].fillna(df['resume_age'].mean())
df['resume_experience_months'] = df['resume_experience_months'].fillna(0)

In [15]:
df = df.drop(['vacancy_salary_to', 'vacancy_salary_from',
              'vacancy_salary_currency', 'vacancy_salary_gross'], axis=1)

In [16]:
df.loc[df['resume_last_company_experience_period'] == 'NDT', 'resume_last_experience_description'].unique()

array(['NDT'], dtype=object)

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">

- Ограничим выбросы по зарплате, потому что ровно одно значение по ожидаемой заработоной плате = 999,999,999 (смешно, но нет)

- Ограничим опыт общий и внутри одной компании до 720 месяцев (60 лет, ничего себе уже)

- Уберем возраст > 90, не ждем, что эти кандидаты находятся в поиске вакансии
</div>

In [26]:
df = df[~(df.resume_salary > 1e7)]
df.loc[df['resume_experience_months'] > 720, 'resume_experience_months'] = 720
df.loc[df['resume_last_company_experience_months'] > 720, 'resume_last_company_experience_months'] = 720
df = df[~(df.resume_age > 90)]

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">

- Также уберем строки, где последний опыт кандидата больше, чем общий

- И где общий опыт кандидата +16 лет больше чем возраст (хоть так)

</div>

In [27]:
df = df[~(df.resume_experience_months < df.resume_last_company_experience_months)]
df = df[~(df.resume_age < (df.resume_experience_months // 12) + 16)]

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">

Заменим текущий формат разброса полов в датасете на унифицированный

</div>

In [29]:
gender_map = {
    'Мужчина': 'Мужчина',
    'Male': 'Мужчина',
    'Женщина': 'Женщина',
    'Female': 'Женщина'
}

df['resume_gender'] = df['resume_gender'].apply(lambda x: gender_map[x] if x in gender_map else 'Неизвестно')

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">
Преобразуем сначала ожидаемые зарплаты
</div>

In [19]:
df['resume_salary_split'] = df['resume_salary'].apply(lambda x: x.split())

df['salary_int'] = df['resume_salary_split'].apply(
    lambda x: int(''.join(part for part in x if re.fullmatch(r'\d+', part)))
              if any(re.fullmatch(r'\d+', part) for part in x)
              else np.nan
)

currency_symbols = ['₽', '$', '€', '₴', '₸', '₼', '₾', 'Br', "so'm"]

rates_rub = {
    "₽": 1.0,
    "$": 80.85,
    "€": 94.14,
    "₴": 1.94,
    "₸": 0.150,
    "₼": 47.8,
    "₾": 33.5,
    "Br": 28.7,
    "so'm": 0.0068
}

df['currency_symbol'] = df['resume_salary_split'].apply(
    lambda x: next((sym for sym in x if sym in currency_symbols), np.nan)
)

df['salary_converted'] = (df['salary_int'] * df['currency_symbol'].map(rates_rub)).fillna(0)

df['resume_salary'] = df['salary_converted']

df = df.drop(['resume_salary_split', 'salary_int', 'currency_symbol', 'salary_converted'], axis=1)

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">
Добавим дополнительный столбец с опытом работы в последней компании в месяцах для удобства
</div>

In [21]:
def experience_to_months(experience_text):
    months = 0
    # Опыт в годах
    years_match = re.search(r'(\d+)\s*год', experience_text)
    if years_match:
        months += int(years_match.group(1)) * 12

    years_match = re.search(r'(\d+)\s*лет', experience_text)
    if years_match:
        months += int(years_match.group(1)) * 12

    # Опыт в месяцах
    months_match = re.search(r'(\d+)\s*месяц', experience_text)
    if months_match:
        months += int(months_match.group(1))

    return months if months > 0 else np.nan

In [22]:
df['resume_last_company_experience_months'] = df['resume_last_company_experience_period'].apply(experience_to_months)

In [23]:
df.loc[df['resume_last_company_experience_period'] == 'NDT', 'resume_last_experience_description'].unique()

array(['NDT'], dtype=object)

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">
Т.к. в названии компании стоит NDT, можно столбец resume_last_company_experience_months заполнять нулевыми значениями.
</div>

In [24]:
df['resume_last_company_experience_months'] = df['resume_last_company_experience_months'].fillna(0)

# Base model

## Train-test split

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">
Выберем признаки для первичного обучения

</div>

In [238]:
features = [
    'vacancy_area',
    'vacancy_experience',
    'vacancy_employment', 
    'vacancy_schedule',
    # 'resume_specialization',
    # 'resume_education', 
    # 'resume_courses', 
    'resume_salary',
    'resume_age', 
    'resume_experience_months',
    'resume_location',
    'resume_gender', 
    'resume_applicant_status', 
    'resume_last_company_experience_months'
]
df[features]

,vacancy_experience,vacancy_employment,vacancy_schedule,resume_salary,resume_age,resume_experience_months,resume_gender,resume_applicant_status,resume_last_company_experience_months,location_matching,resume_skill_count_in_vacancy,last_position_in_vacancy,similarity_score_tfidf
0,Более 6 лет,Полная занятость,Удаленная работа,0.0,65.000000,228.0,Мужчина,Рассматривает предложения,76.0,1,3,0.666667,0.284047
1,Более 6 лет,Полная занятость,Удаленная работа,0.0,43.000000,208.0,Мужчина,Рассматривает предложения,8.0,1,2,0.500000,0.308726
2,Более 6 лет,Полная занятость,Удаленная работа,200000.0,52.000000,360.0,Женщина,NDT,136.0,1,1,0.000000,0.510093
3,Более 6 лет,Полная занятость,Удаленная работа,500000.0,56.000000,356.0,Мужчина,Рассматривает предложения,135.0,0,2,0.333333,0.301062
4,Более 6 лет,Полная занятость,Удаленная работа,0.0,48.000000,301.0,Мужчина,NDT,0.0,0,2,0.600000,0.075429
...,...,...,...,...,...,...,...,...,...,...,...,...,...
325538,От 3 до 6 лет,Полная занятость,Удаленная работа,242550.0,66.000000,521.0,Женщина,NDT,270.0,0,0,0.166667,0.072670
325539,От 3 до 6 лет,Полная занятость,Удаленная работа,0.0,40.000000,213.0,Мужчина,Активно ищет работу,35.0,1,0,0.000000,0.000000
325540,От 3 до 6 лет,Полная занятость,Удаленная работа,80000.0,44.060813,121.0,Мужчина,NDT,44.0,1,0,0.200000,0.047398
325541,От 3 до 6 лет,Полная занятость,Удаленная работа,0.0,32.000000,117.0,Женщина,NDT,96.0,1,0,0.200000,0.029086


In [239]:
numeric_features = df[features].select_dtypes(include=np.number).columns
categorical_features = df[features].select_dtypes(exclude=np.number).columns

In [270]:
X = df[features]
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

In [271]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((260434, 13), (65109, 13), (260434,), (65109,))

## Column Transformer

In [273]:
col_transformer = ColumnTransformer([
        ('numeric_scaling', StandardScaler(), numeric_features),
        ('categorical_encoding', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ])

In [274]:
X_train_transformed = col_transformer.fit_transform(X_train)
X_test_transformed = col_transformer.transform(X_test)

## Train model

In [275]:
lr = LogisticRegression(random_state=RANDOM_STATE)
lr.fit(X_train_transformed, y_train)
y_pred_proba = lr.predict_proba(X_test_transformed)

df_test = df.loc[X_test.index]
df_test['y_pred_proba'] = y_pred_proba[:, 1]

In [272]:
def calculate_metrics(df_test: pd.DataFrame) -> pd.DataFrame:
    ndcg_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    vacancy_ids = df_test['vacancy_id'].unique()
    
    for vacancy_id in vacancy_ids:
        mask = df_test['vacancy_id'] == vacancy_id
        y_true = df_test.loc[mask, 'target'].values
        y_score = df_test.loc[mask, 'y_pred_proba'].values
        
        if len(y_true) <= 1:
            continue
        
        y_true_2d = y_true.reshape(1, -1)
        y_score_2d = y_score.reshape(1, -1)
        
        ndcg = ndcg_score(y_true_2d, y_score_2d)
        ndcg_scores.append(ndcg)
        
        y_pred_binary = (y_score >= 0.5).astype(int)
        
        precision = precision_score(y_true, y_pred_binary, zero_division=0)
        recall = recall_score(y_true, y_pred_binary, zero_division=0)
        f1 = f1_score(y_true, y_pred_binary, zero_division=0)
        
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
    
    if ndcg_scores:
        print(f"Средний NDCG: {np.mean(ndcg_scores):.4f}")
        print(f"Средний Precision: {np.mean(precision_scores):.4f}")
        print(f"Средний Recall: {np.mean(recall_scores):.4f}")
        print(f"Средний F1-Score: {np.mean(f1_scores):.4f}")

        return np.mean(ndcg_scores), np.mean(precision_scores), np.mean(recall_scores), np.mean(f1_scores)
    else:
        print("Недостаточно данных для расчета метрик")

        return None, None, None, None

In [276]:
ndcg, precision, recall, f1 = calculate_metrics(df_test)
metrics_baseline = {}
metrics_baseline['ndcg'] = ndcg
metrics_baseline['precision'] = precision
metrics_baseline['recall'] = recall
metrics_baseline['f1'] = f1

Средний NDCG: 0.7371
Средний Precision: 0.6091
Средний Recall: 0.5684
Средний F1-Score: 0.5734


In [265]:
best_params = lr.get_params()

In [266]:
RUN_NAME = "base_line" 
REGISTRY_MODEL_NAME = "base_line_model"

In [268]:
signature = mlflow.models.infer_signature(X_test, y_test)
input_example = X_test_transformed[:10]
code_paths = ["linear_models.ipynb"]

try:
    experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
except:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    lr_info = mlflow.sklearn.log_model(sk_model=lr, 
                                       artifact_path='base_line_model',
                                       registered_model_name=REGISTRY_MODEL_NAME,
                                       input_example=input_example,
                                       code_paths=code_paths,
                                       await_registration_for=60
                                      )
    mlflow.log_metrics(metrics_baseline)
    mlflow.log_params(best_params)

Successfully registered model 'base_model'.
2025/11/19 16:51:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 60 seconds for model version to finish creation. Model name: base_model, version 1


🏃 View run base_model at: http://127.0.0.1:5000/#/experiments/1/runs/dbfc86766916445bbbaf5a2fd5219e63
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Created version '1' of model 'base_model'.


# Feature engineering

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">
Добавим признак матчинга локации вакансии и резюме

</div>

In [85]:
df['location_matching'] = df.apply(lambda row: 1 if row['vacancy_area'] == row['resume_location'] else 0, axis=1)

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">

Сделаем новый признак, а именно посчитаем количество навыков кандидата, которые указаны в вакансии.

</div>

In [86]:
def resume_skill_count_in_vacancy(row):
    count = 0
    skill_list = row['resume_skills'].replace('[', '').replace(']', '').replace("'", "").split(', ')
    for i in skill_list:
        if i in row['vacancy_description']:
            count += 1
    return count

df['resume_skill_count_in_vacancy'] = df.apply(resume_skill_count_in_vacancy, axis=1)

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">

Также посчитаем долю слов из последней должности в резюме, которые указаны в вакансии.

</div>

In [76]:
def last_position_in_vacancy(row):
    bow = []
    seps = [' ', '-', '_']
    for sep in seps:
        bow += row['resume_last_position'].split(sep=sep)
        bow = list(set(bow))
    
    c = 0
    for word in bow:
        if word in row['vacancy_description']:
            c +=1
    
    return c / len(bow)

In [79]:
df['last_position_in_vacancy'] = df.apply(last_position_in_vacancy, axis=1)

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">

Теперь закодируем описание вакансии и последнего опыта работы разными способами и сравним через косинусное расстояние.

</div>

In [32]:
def preprocess_data(df):
    """Обработка пропущенных значений в текстовых полях"""
    print("Проверка пропущенных значений...")
    print(f"Пропуски в vacancy_description: {df['vacancy_description'].isna().sum()}")
    print(f"Пропуски в resume_last_experience_description: {df['resume_last_experience_description'].isna().sum()}")
    
    # Заполняем пропуски пустыми строками
    df['vacancy_description'] = df['vacancy_description'].fillna('')
    df['resume_last_experience_description'] = df['resume_last_experience_description'].fillna('')
    
    # Проверяем, что все значения теперь строковые
    df['vacancy_description'] = df['vacancy_description'].astype(str)
    df['resume_last_experience_description'] = df['resume_last_experience_description'].astype(str)
    
    return df

In [33]:
def save_results(df, output_file):
    """Сохранение результатов в CSV файл"""
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Результаты сохранены в файл: {output_file}")

In [34]:
def calculate_cosine_similarity(embeddings1, embeddings2):
    """Вычисление косинусного сходства между двумя наборами эмбеддингов"""
    similarities = []
    
    for i in tqdm(range(embeddings1.shape[0])):
        emb1_row = embeddings1[i]
        emb2_row = embeddings2[i]
        
        similarity = cosine_similarity(emb1_row, emb2_row)[0][0]
        similarities.append(similarity)
    
    return similarities

In [35]:
warnings.filterwarnings('ignore')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('taggers/averaged_perceptron_tagger_ru')
except LookupError:
    nltk.download('averaged_perceptron_tagger_ru')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

morph = pymorphy3.MorphAnalyzer()

[nltk_data] Downloading package wordnet to /Users/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
def lemmatize_russian(tokens):
    """Лемматизация русских слов"""
    lemmas = []
    for token in tokens:
        parsed = morph.parse(token)[0]  # Берем самый вероятный разбор
        lemmas.append(parsed.normal_form)
    return lemmas

In [37]:
def tokenize_and_lemmatize(text):
    """Токенизация текста с лемматизацией и удалением стоп-слов"""
    # Базовая токенизация
    tokens = simple_preprocess(text, deacc=True, min_len=2)
    
    # Удаляем стоп-слова
    stop_words = set(stopwords.words('russian') + stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Лемматизация для русских слов
    lemmatized_tokens = lemmatize_russian(tokens)
    
    return lemmatized_tokens

In [44]:
def get_tfidf_embeddings(texts, vectorizer=None, fit=True):
    """Создание TF-IDF эмбеддингов для списка текстов с лемматизацией"""
    if fit:
        vectorizer = TfidfVectorizer(
            max_features=5000,
            min_df=2,
            max_df=0.8,
            ngram_range=(1, 2),
            tokenizer=tokenize_and_lemmatize,
            token_pattern=None,
            lowercase=False  # Уже сделано в токенизации
        )
        embeddings = vectorizer.fit_transform(texts)
    else:
        embeddings = vectorizer.transform(texts)
    
    return embeddings, vectorizer

In [45]:
def get_tfidf_vacancy_embeddings(df, vectorizer=None):
    """Создание эмбеддингов для уникальных вакансий с лемматизацией"""
    unique_vacancies = df[['vacancy_id', 'vacancy_description']].drop_duplicates()
    
    unique_embeddings, vectorizer = get_tfidf_embeddings(
        unique_vacancies['vacancy_description'].tolist(), 
        vectorizer=vectorizer, 
        fit=(vectorizer is None)
    )
    
    # Создаем mapping: vacancy_id -> sparse row
    vacancy_embedding_dict = dict(zip(unique_vacancies['vacancy_id'], unique_embeddings))
    
    rows = []
    for vid in df['vacancy_id']:
        rows.append(vacancy_embedding_dict[vid])
    
    # Объединяем в одну sparse матрицу
    all_vacancy_embeddings = vstack(rows)
    
    return all_vacancy_embeddings, vectorizer

In [46]:
def process_similarity_scores_tfidf(df, vectorizer=None, fit=True):
    """Функция для вычисления схожести с использованием TF-IDF и лемматизации"""    
    # Предобработка данных
    df = preprocess_data(df)
    
    print("Создание TF-IDF эмбеддингов для описаний опыта в резюме...")
    experience_embeddings, tfidf_vectorizer = get_tfidf_embeddings(df['resume_last_experience_description'].tolist(), vectorizer=vectorizer, fit=fit)
    
    print("Создание TF-IDF эмбеддингов для описаний вакансий...")
    vacancy_embeddings, _ = get_tfidf_vacancy_embeddings(df, vectorizer=tfidf_vectorizer)
    
    print("Вычисление косинусного сходства...")
    similarity_scores = calculate_cosine_similarity(vacancy_embeddings, experience_embeddings)
    
    # Добавляем scores в DataFrame
    df['similarity_score_tfidf'] = similarity_scores
    
    return df, tfidf_vectorizer

In [50]:
df_tfidf = process_similarity_scores_tfidf(df.copy())
save_results(df_tfidf, 'description_df_with_scores_tfidf.csv')

Проверка пропущенных значений...
Пропуски в vacancy_description: 0
Пропуски в resume_last_experience_description: 0
Создание TF-IDF эмбеддингов для описаний опыта в резюме...
Создание TF-IDF эмбеддингов для описаний вакансий...
Вычисление косинусного сходства...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 325543/325543 [01:04<00:00, 5032.68it/s]


Результаты сохранены в файл: description_df_with_scores_tfidf.csv


In [52]:
df = df.merge(df_tfidf)

In [30]:
df

,vacancy_id,vacancy_name,vacancy_area,vacancy_experience,vacancy_employment,vacancy_schedule,vacancy_description,resume_id,resume_title,resume_specialization,...,resume_education,resume_courses,resume_salary,resume_age,resume_experience_months,resume_location,resume_gender,resume_applicant_status,target,resume_last_company_experience_months
0,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,"Привет!.redev — технологическая компания, созд...",6969174,ABAP-разработчик,"['Программист, разработчик']",...,['Казанский Авиационный Институт'],NDT,0.0,65.000000,228.0,Москва,Мужчина,Рассматривает предложения,1,76.0
1,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,"Привет!.redev — технологическая компания, созд...",9100077,"ABAP разработчик - SAP HCM, CRM, S/4HANA ERP(F...","['Программист, разработчик']",...,"['ООО ""Открытый Учебный Центр СофтБаланс"", г. ...","['ООО ""Открытый Учебный Центр СофтБаланс"", г. ...",0.0,43.000000,208.0,Москва,Мужчина,Рассматривает предложения,1,8.0
2,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,"Привет!.redev — технологическая компания, созд...",32644957,Разработчик ABAP,"['Программист, разработчик']",...,['Орский государственный педагогический инстит...,NDT,200000.0,52.000000,360.0,Москва,Женщина,NDT,1,136.0
3,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,"Привет!.redev — технологическая компания, созд...",27220466,ABAP-разработчик,"['Программист, разработчик']",...,['Красноярский государственный университет'],NDT,500000.0,56.000000,356.0,Красноярск,Мужчина,Рассматривает предложения,1,135.0
4,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,"Привет!.redev — технологическая компания, созд...",7532708,ABAP разработчик. Senior ABAP Developer. SAP T...,"['Programmer, developer']",...,['Белоруский Гос. Университет Информатики и Ра...,"['SAP CIS, SAP XI', 'Школа Логистики МАДИ', 'S...",0.0,48.000000,301.0,Moscow,Мужчина,NDT,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332325,125547086,Golang developer (IT банк),Москва,От 3 до 6 лет,Полная занятость,Удаленная работа,Мы — Wildberries. Мы — это крупнейший маркетпл...,255135,"Управляющий недвижимостью, Директор по управле...",['Агент по недвижимости'],...,['Ленинградский Электротехнический Институт им...,['Национальный Колледж Оценки и Управления Нед...,242550.0,66.000000,521.0,Санкт-Петербург,Женщина,NDT,0,270.0
332326,125547086,Golang developer (IT банк),Москва,От 3 до 6 лет,Полная занятость,Удаленная работа,Мы — Wildberries. Мы — это крупнейший маркетпл...,34423776,"Директор, Операционный директор,Технический ди...","['Механик', 'Технический директор (CTO)']",...,['Кузбасский государственный технический униве...,NDT,0.0,40.000000,213.0,Москва,Мужчина,Активно ищет работу,0,35.0
332327,125547086,Golang developer (IT банк),Москва,От 3 до 6 лет,Полная занятость,Удаленная работа,Мы — Wildberries. Мы — это крупнейший маркетпл...,54564265,Интернет-маркетолог,"['Менеджер по маркетингу, интернет-маркетолог']",...,['Московский финансово-промышленный университе...,['Точка Кода - Всероссийская школа программиро...,80000.0,44.060813,121.0,Москва,Мужчина,NDT,0,44.0
332328,125547086,Golang developer (IT банк),Москва,От 3 до 6 лет,Полная занятость,Удаленная работа,Мы — Wildberries. Мы — это крупнейший маркетпл...,70588832,Менеджер по продукту,"['Менеджер продукта', 'Руководитель проектов']",...,"['Национальный исследовательский университет ""...","['ECCN2017, Poland, Warsaw, Speaker', 'Max Pla...",0.0,32.000000,117.0,Москва,Женщина,NDT,0,96.0


In [238]:
features = [
    'vacancy_area',
    'vacancy_experience',
    'vacancy_employment', 
    'vacancy_schedule',
    # 'resume_specialization',
    # 'resume_education', 
    # 'resume_courses', 
    'resume_salary',
    'resume_age', 
    'resume_experience_months',
    'resume_location',
    'resume_gender', 
    'resume_applicant_status', 
    'resume_last_company_experience_months', 
    'location_matching',
    'resume_skill_count_in_vacancy',
    'last_position_in_vacancy',
    'similarity_score_tfidf'
]
df[features]

,vacancy_experience,vacancy_employment,vacancy_schedule,resume_salary,resume_age,resume_experience_months,resume_gender,resume_applicant_status,resume_last_company_experience_months,location_matching,resume_skill_count_in_vacancy,last_position_in_vacancy,similarity_score_tfidf
0,Более 6 лет,Полная занятость,Удаленная работа,0.0,65.000000,228.0,Мужчина,Рассматривает предложения,76.0,1,3,0.666667,0.284047
1,Более 6 лет,Полная занятость,Удаленная работа,0.0,43.000000,208.0,Мужчина,Рассматривает предложения,8.0,1,2,0.500000,0.308726
2,Более 6 лет,Полная занятость,Удаленная работа,200000.0,52.000000,360.0,Женщина,NDT,136.0,1,1,0.000000,0.510093
3,Более 6 лет,Полная занятость,Удаленная работа,500000.0,56.000000,356.0,Мужчина,Рассматривает предложения,135.0,0,2,0.333333,0.301062
4,Более 6 лет,Полная занятость,Удаленная работа,0.0,48.000000,301.0,Мужчина,NDT,0.0,0,2,0.600000,0.075429
...,...,...,...,...,...,...,...,...,...,...,...,...,...
325538,От 3 до 6 лет,Полная занятость,Удаленная работа,242550.0,66.000000,521.0,Женщина,NDT,270.0,0,0,0.166667,0.072670
325539,От 3 до 6 лет,Полная занятость,Удаленная работа,0.0,40.000000,213.0,Мужчина,Активно ищет работу,35.0,1,0,0.000000,0.000000
325540,От 3 до 6 лет,Полная занятость,Удаленная работа,80000.0,44.060813,121.0,Мужчина,NDT,44.0,1,0,0.200000,0.047398
325541,От 3 до 6 лет,Полная занятость,Удаленная работа,0.0,32.000000,117.0,Женщина,NDT,96.0,1,0,0.200000,0.029086


In [239]:
numeric_features = df[features].select_dtypes(include=np.number).columns
categorical_features = df[features].select_dtypes(exclude=np.number).columns

## Train-test split

In [270]:
X = df[features]
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

In [271]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((260434, 13), (65109, 13), (260434,), (65109,))

# Base pipeline

In [273]:
col_transformer = ColumnTransformer([
        ('numeric_scaling', StandardScaler(), numeric_features),
        ('categorical_encoding', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ])

In [274]:
X_train_transformed = col_transformer.fit_transform(X_train)
X_test_transformed = col_transformer.transform(X_test)

In [275]:
lr = LogisticRegression(random_state=RANDOM_STATE)
lr.fit(X_train_transformed, y_train)
y_pred_proba = lr.predict_proba(X_test_transformed)

df_test = df.loc[X_test.index]
df_test['y_pred_proba'] = y_pred_proba[:, 1]

In [272]:
def calculate_metrics(df_test: pd.DataFrame) -> pd.DataFrame:
    ndcg_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    vacancy_ids = df_test['vacancy_id'].unique()
    
    for vacancy_id in vacancy_ids:
        mask = df_test['vacancy_id'] == vacancy_id
        y_true = df_test.loc[mask, 'target'].values
        y_score = df_test.loc[mask, 'y_pred_proba'].values
        
        if len(y_true) <= 1:
            continue
        
        y_true_2d = y_true.reshape(1, -1)
        y_score_2d = y_score.reshape(1, -1)
        
        ndcg = ndcg_score(y_true_2d, y_score_2d)
        ndcg_scores.append(ndcg)
        
        y_pred_binary = (y_score >= 0.5).astype(int)
        
        precision = precision_score(y_true, y_pred_binary, zero_division=0)
        recall = recall_score(y_true, y_pred_binary, zero_division=0)
        f1 = f1_score(y_true, y_pred_binary, zero_division=0)
        
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
    
    if ndcg_scores:
        print(f"Средний NDCG: {np.mean(ndcg_scores):.4f}")
        print(f"Средний Precision: {np.mean(precision_scores):.4f}")
        print(f"Средний Recall: {np.mean(recall_scores):.4f}")
        print(f"Средний F1-Score: {np.mean(f1_scores):.4f}")

        return np.mean(ndcg_scores), np.mean(precision_scores), np.mean(recall_scores), np.mean(f1_scores)
    else:
        print("Недостаточно данных для расчета метрик")

        return None, None, None, None

In [276]:
ndcg, precision, recall, f1 = calculate_metrics(df_test)
metrics_baseline = {}
metrics_baseline['ndcg'] = ndcg
metrics_baseline['precision'] = precision
metrics_baseline['recall'] = recall
metrics_baseline['f1'] = f1

Средний NDCG: 0.7371
Средний Precision: 0.6091
Средний Recall: 0.5684
Средний F1-Score: 0.5734


In [265]:
best_params = lr.get_params()

In [266]:
RUN_NAME = "base_model" 
REGISTRY_MODEL_NAME = "base_model"

In [268]:
signature = mlflow.models.infer_signature(X_test, y_test)
input_example = X_test_transformed[:10]
code_paths = ["linear_models.ipynb"]

try:
    experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
except:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    lr_info = mlflow.sklearn.log_model(sk_model=lr, 
                                       artifact_path='base_model',
                                       registered_model_name=REGISTRY_MODEL_NAME,
                                       input_example=input_example,
                                       code_paths=code_paths,
                                       await_registration_for=60
                                      )
    mlflow.log_metrics(metrics_baseline)
    mlflow.log_params(best_params)

Successfully registered model 'base_model'.
2025/11/19 16:51:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 60 seconds for model version to finish creation. Model name: base_model, version 1


🏃 View run base_model at: http://127.0.0.1:5000/#/experiments/1/runs/dbfc86766916445bbbaf5a2fd5219e63
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Created version '1' of model 'base_model'.


In [278]:
from sklearn import set_config
set_config(display='diagram')

pipeline = Pipeline([
    ('preprocessing', ColumnTransformer([
        ('numeric_scaling', StandardScaler(), numeric_features),
        ('categorical_encoding', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ])),
    ('model', LogisticRegression(random_state=RANDOM_STATE))
])

pipeline.fit(X_train, y_train)

,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('numeric_scaling', ...), ('categorical_encoding', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [279]:
y_pred_proba = pipeline.predict_proba(X_test)

In [280]:
y_pred_proba[:, 1]

array([0.00294784, 0.00319049, 0.00142391, ..., 0.00262649, 0.00302502,
       0.00397595], shape=(65109,))

In [281]:
df_test = df.loc[X_test.index]
df_test['y_pred_proba'] = y_pred_proba[:, 1]

In [282]:
def calculate_metrics(df_test: pd.DataFrame) -> pd.DataFrame:
    ndcg_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    vacancy_ids = df_test['vacancy_id'].unique()
    
    for vacancy_id in vacancy_ids:
        mask = df_test['vacancy_id'] == vacancy_id
        y_true = df_test.loc[mask, 'target'].values
        y_score = df_test.loc[mask, 'y_pred_proba'].values
        
        if len(y_true) <= 1:
            continue
        
        y_true_2d = y_true.reshape(1, -1)
        y_score_2d = y_score.reshape(1, -1)
        
        ndcg = ndcg_score(y_true_2d, y_score_2d)
        ndcg_scores.append(ndcg)
        
        y_pred_binary = (y_score >= 0.5).astype(int)
        
        precision = precision_score(y_true, y_pred_binary, zero_division=0)
        recall = recall_score(y_true, y_pred_binary, zero_division=0)
        f1 = f1_score(y_true, y_pred_binary, zero_division=0)
        
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
    
    if ndcg_scores:
        print(f"Средний NDCG: {np.mean(ndcg_scores):.4f}")
        print(f"Средний Precision: {np.mean(precision_scores):.4f}")
        print(f"Средний Recall: {np.mean(recall_scores):.4f}")
        print(f"Средний F1-Score: {np.mean(f1_scores):.4f}")

        return np.mean(ndcg_scores), np.mean(precision_scores), np.mean(recall_scores), np.mean(f1_scores)
    else:
        print("Недостаточно данных для расчета метрик")

        return None, None, None, None

In [283]:
ndcg, precision, recall, f1 = calculate_metrics(df_test)
metrics_baseline = {}
metrics_baseline['ndcg'] = ndcg
metrics_baseline['precision'] = precision
metrics_baseline['recall'] = recall
metrics_baseline['f1'] = f1

Средний NDCG: 0.7371
Средний Precision: 0.6091
Средний Recall: 0.5684
Средний F1-Score: 0.5734


In [ ]:
RUN_NAME = "base_pip" 
REGISTRY_MODEL_NAME = "base_model"